In [7]:
import cv2
import numpy as np

cap = cv2.VideoCapture("C:/Users/Shreyas/Downloads/Ass_1/video.mp4")

blue_lower = (100, 150, 150)
blue_upper = (140, 255, 255)
yellow_lower = (20, 100, 100)
yellow_upper = (30, 255, 255)
red_lower = (0, 100, 100)
red_upper = (13, 255, 255)
detected_centers = []
def stack_images(images):
  res = np.hstack(images)
  return res

def find_archery_center(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    height = frame.shape[0]
    crop_height = int(height * 0.65)
    frame = frame[:crop_height, :]
    
    blue_mask = cv2.inRange(hsv, blue_lower, blue_upper)
    yellow_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)
    red_mask = cv2.inRange(hsv, red_lower, red_upper)

    kernel = np.ones((5, 5), np.uint8)
    blue_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_CLOSE, kernel)
    yellow_mask = cv2.morphologyEx(yellow_mask, cv2.MORPH_CLOSE, kernel)
    red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel)

    blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    yellow_contours, _ = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    red_contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    def filter_contours(contours, min_area=200):
        return [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

    def filter_aspect_ratio(contours):
        filtered_contours = []
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            aspect_ratio = float(w) / h
            if 0.5 <= aspect_ratio <= 1.5:
                filtered_contours.append(cnt)
        return filtered_contours

    blue_contours = filter_contours(blue_contours)
    yellow_contours = filter_contours(yellow_contours)
    red_contours = filter_aspect_ratio(filter_contours(red_contours))  
    detected_centers = []
    if blue_contours:
        largest_blue_contour = max(blue_contours, key=cv2.contourArea)
        blue_center = (cv2.moments(largest_blue_contour)['m10'] / cv2.moments(largest_blue_contour)['m00'],
                    cv2.moments(largest_blue_contour)['m01'] / cv2.moments(largest_blue_contour)['m00'])
        cv2.circle(frame, (int(blue_center[0]), int(blue_center[1])), 6, (255, 0, 0), -1)
        detected_centers.append(blue_center)
        
    if yellow_contours:
        largest_yellow_contour = max(yellow_contours, key=cv2.contourArea)
        yellow_center = (cv2.moments(largest_yellow_contour)['m10'] / cv2.moments(largest_yellow_contour)['m00'],
                    cv2.moments(largest_yellow_contour)['m01'] / cv2.moments(largest_yellow_contour)['m00'])
        cv2.circle(frame, (int(yellow_center[0]), int(yellow_center[1])), 6, (0, 255, 255), -1)
        detected_centers.append(yellow_center)

    if red_contours:
        largest_red_contour = max(red_contours, key=cv2.contourArea)
        red_center = (cv2.moments(largest_red_contour)['m10'] / cv2.moments(largest_red_contour)['m00'],
                    cv2.moments(largest_red_contour)['m01'] / cv2.moments(largest_red_contour)['m00'])
        cv2.circle(frame, (int(red_center[0]), int(red_center[1])), 6, (0, 0, 255), -1)
        detected_centers.append(red_center)

    if (len(detected_centers) == 1):
        archery_center = detected_centers[0]
    elif (len(detected_centers) == 2):
        archery_center = ((detected_centers[0][0]+detected_centers[1][0])/2,(detected_centers[0][1]+detected_centers[1][1])/2)
    elif (len(detected_centers) == 3):
        archery_center = ((detected_centers[0][0]+detected_centers[1][0]+detected_centers[2][0])/3,(detected_centers[0][1]+detected_centers[1][1]+detected_centers[2][1])/3)  
    else:
        archery_center = None
    cv2.circle(frame, (int(archery_center[0]), int(archery_center[1])), 6, (0, 255, 0), -1)

    return frame, archery_center, blue_contours, yellow_contours, red_contours
while True:
    ret, frame = cap.read()
    resize_ratio = 0.5
    frame = cv2.resize(frame, (int(frame.shape[1] * resize_ratio), int(frame.shape[0] * resize_ratio)))
    if not ret:
        break
    frame_count = cap.get(cv2.CAP_PROP_POS_FRAMES) - 1
    frame, center, blue_contours, yellow_contours, red_contours = find_archery_center(frame)

    blue_contour_img = np.zeros_like(frame)
    yellow_contour_img = np.zeros_like(frame)
    red_contour_img = np.zeros_like(frame)
    cv2.drawContours(blue_contour_img, blue_contours, -1, (255, 0, 0), 2)
    cv2.drawContours(yellow_contour_img, yellow_contours, -1, (0, 255, 255), 2)
    cv2.drawContours(red_contour_img, red_contours, -1, (0, 0, 255), 2)

    stacked_image = stack_images([frame, blue_contour_img, yellow_contour_img, red_contour_img])
    cv2.imshow('Archery Board Analysis', stacked_image)

    if cv2.waitKey(500) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()